# RNN sequence to sequence範例
在這個筆記本我們會建立一個能夠讀取一整句文字的模型，輸出則是被排序好的句子。


## Dataset 

#### 英文/ 法文的對應文黨 fra.txt
1. Go.	Va
2. Hi.	Salut
3. ...

In [1]:
from __future__ import print_function

from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, LSTM, Dense

import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'Data/fra.txt'

## 打開文件並讀取資料
可以看到資料是英文與法文的轉換

In [2]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
for line in lines[: min(num_samples, len(lines) - 1)]:

    a,b,c = line.split('\t')
    print('Eng:',a,'\tFra:',b)
    target_text = b 
    input_text = a
    # 我們使用tab分隔文字
    target_text = '\t' + target_text + '\n'
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    # 搜尋文件所有的字元種類
        # 從輸入資料:
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
        # 從目標資料:
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

Eng: Go. 	Fra: Va !
Eng: Hi. 	Fra: Salut !
Eng: Hi. 	Fra: Salut.
Eng: Run! 	Fra: Cours !
Eng: Run! 	Fra: Courez !
Eng: Who? 	Fra: Qui ?
Eng: Wow! 	Fra: Ça alors !
Eng: Fire! 	Fra: Au feu !
Eng: Help! 	Fra: À l'aide !
Eng: Jump. 	Fra: Saute.
Eng: Stop! 	Fra: Ça suffit !
Eng: Stop! 	Fra: Stop !
Eng: Stop! 	Fra: Arrête-toi !
Eng: Wait! 	Fra: Attends !
Eng: Wait! 	Fra: Attendez !
Eng: Go on. 	Fra: Poursuis.
Eng: Go on. 	Fra: Continuez.
Eng: Go on. 	Fra: Poursuivez.
Eng: Hello! 	Fra: Bonjour !
Eng: Hello! 	Fra: Salut !
Eng: I see. 	Fra: Je comprends.
Eng: I try. 	Fra: J'essaye.
Eng: I won! 	Fra: J'ai gagné !
Eng: I won! 	Fra: Je l'ai emporté !
Eng: I won. 	Fra: J’ai gagné.
Eng: Oh no! 	Fra: Oh non !
Eng: Attack! 	Fra: Attaque !
Eng: Attack! 	Fra: Attaquez !
Eng: Cheers! 	Fra: Santé !
Eng: Cheers! 	Fra: À votre santé !
Eng: Cheers! 	Fra: Merci !
Eng: Cheers! 	Fra: Tchin-tchin !
Eng: Get up. 	Fra: Lève-toi.
Eng: Go now. 	Fra: Va, maintenant.
Eng: Go now. 	Fra: Allez-y maintenant.
Eng: Go now.

## 將dataset整理
1. 取得字符表
2. 製造單字矩陣
3. 把所有單字根據字符表填入對應的單字矩陣裡面


In [3]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

# 取得字符表
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

# 製造單字矩陣

# encoder_input_data是3維矩陣，分別是 總共有幾筆資料(10000),
# 資料中最長的seq有多長, 在input_texts忠有幾個獨立的字元
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
# decoder_input_data，分別是 總共有幾筆資料(10000), 資料中最長的seq有多長,
# 在target_texts中有幾個獨立的字元
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
# decoder_target_data，分別是 總共有幾筆資料(10000), 資料中最長的seq有多長,
# 在target_texts中有幾個獨立的字元
# 他與上面的decoder_input_data的shape是一模一樣的
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

# 把所有單字根據字符表填入對應的單字矩陣裡面

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        # input_token_index[char]會給出char這個字元在input_characters這個矩陣的位置，
        # 這邊以類似onehot的方式來對encoder_input_data編碼。
        encoder_input_data[i, t, input_token_index[char]] = 1.
# encoder_input_data[i, t, input_token_index[char]]
# i:第i筆資料(0~9999)
# t:該筆資料的第幾個單字 (ex: Go的G為0, o為1)
# input_token_index[char]: 該單字是什麼單字 (ex: A~z)

    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data會在one timestep之前，並且不包含起始字元
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# print(input_token_index)
print(decoder_target_data.shape)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59
(10000, 59, 93)


### 我們看一下被轉化後的input data和decoder data
#### 英文部分(input text)一句話最常有16個字母，所以是16個array, 每個array長度70(法文的字母表長度)
#### 法文部分(output text)一句話最常有59個字母，所以是59個array, 每個array長度93(法文的字母表長度)
- 下面為input text被轉化進encoder_input_data的示意圖
<img src='images/字母表.png'/>


In [4]:
print(encoder_input_data.shape)
print(decoder_input_data.shape)

(10000, 16, 70)
(10000, 59, 93)


In [62]:
decoder_input_data[0][4]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [61]:
decoder_target_data[0][3]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

### Note: 舉例來說 "Va !" 對字母表而言，分別是 "\t", "V", "a", " ", "!" 和後面的換行省略字元。
同理: "Go." 在字母表上則是 "\t", "G", "o", "." 和後面的換行省略字元。

<img src='images/LSTM_seq2seq.png'/>
<br>

## 客製化model (Keras functional API)
之前我們都是使用sequential model，但今天要做的sequence to sequence (seq2seq)模型比較複雜，要用客製化的方式，也就是Keras functional API來建立模型。
<br>
---
### 流程如下:

#### STEP1. 將英文資料輸入LSTM layer(encoder)中。 我們在前面製作了字母表將單字換成了向量，現在我們需要知道單字和單字之間的距離關係。
- **我們只需要把隱藏層的hidden state傳下去，output是英文，我們不需要。**

#### STEP2. 將法文資料輸入LSTM layer(encoder)中。 我們在前面製作了字母表將單字換成了向量，現在我們需要知道單字和單字之間的距離關係。
- **前一層LSTM隱藏層的hidden state會傳到這層LSTM裡，而這層的output是法文，就是我們需要的輸出。(隱藏層則不需要繼續傳遞了)**

#### STEP3. 在法文輸出的LSTM layer後，我們加上一層Dense layer(單純神經元)，神經元的個數相當於法文的字符種類數量，並使用softmax激活。
- **這是為了讓讓每個神經元的輸出總和為1，並藉此擬似出每個字元被使用機率，取其最高者。**

**-----------------以上完成模型的建立，接著進行compile與fit。------------------**

#### STEP4. 因為輸出是不同的字元，所以loss使用'categorical_crossentropy'來進行compile。
- **使用accuracy評估**

#### STEP5. fit的時候要分別輸入英文資料(encoder_input_data)與法文資料(decoder_input_data)。y則是decoder_target_data。
- **validation_split=0.2是用來做train_valid測試**

In [5]:
# early stoppping

# Step1
# ----------------------------------------------------------------------------------------------------
# 定義一個輸入的sequence框架
encoder_inputs = Input(shape=(None, num_encoder_tokens))

encoder = LSTM(units=latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# 忽略掉輸入的output, 只保留state
encoder_states = [state_h, state_c]
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Step2
# ----------------------------------------------------------------------------------------------------
# 把encoder 的 hidden state 作為 decoder的 hidden state初始狀態
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# 讓decoder回傳整個輸出的sequence, 並且同樣回傳 hiiden state的狀態，雖然在
# 訓練時用不到，但在Inference時會使用

decoder_lstm = LSTM(units=latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
# Step3
# ----------------------------------------------------------------------------------------------------
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# model會把輸入的`encoder_input_data` & `decoder_input_data`轉換成`decoder_target_data`
model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# 進行training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
model.fit(x=[encoder_input_data, decoder_input_data], y=decoder_target_data,
          batch_size=batch_size,
          epochs=epochs)
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# 儲存model
# model.save('s2s.h5')

Train on 10000 samples
Epoch 1/100
10000/10000 [==============================] - 18s 2ms/sample - loss: 1.1656 - accuracy: 0.7206
Epoch 2/100
10000/10000 [==============================] - 14s 1ms/sample - loss: 0.8071 - accuracy: 0.7801
Epoch 3/100
10000/10000 [==============================] - 14s 1ms/sample - loss: 0.6398 - accuracy: 0.8153
Epoch 4/100
10000/10000 [==============================] - 14s 1ms/sample - loss: 0.5675 - accuracy: 0.8337
Epoch 5/100
10000/10000 [==============================] - 14s 1ms/sample - loss: 0.5211 - accuracy: 0.8467
Epoch 6/100
10000/10000 [==============================] - 14s 1ms/sample - loss: 0.4833 - accuracy: 0.8571
Epoch 7/100
10000/10000 [==============================] - 14s 1ms/sample - loss: 0.4541 - accuracy: 0.8649
Epoch 8/100
10000/10000 [==============================] - 14s 1ms/sample - loss: 0.4299 - accuracy: 0.8714
Epoch 9/100
10000/10000 [==============================] - 14s 1ms/sample - loss: 0.4089 - accuracy: 0.8775
Epoch

In [37]:
# Next: inference(推論) mode (sampling).
# 流程如下
# 1) 把輸入投入encoder，取得decoder hidden state的初始狀態
# 2) 讓decoder依據初始狀態跑一個step, 而且一個 "start of sequence"(起始字元)會做為目標。
# 並且，輸出會是下一個target token
# 3) 依據當前的current target token 和 current states重複這個流程

# 定義sampling模型: 
# -------------------
# encoder_model: 輸入是英文單字組成的字母表，輸出是[state_h, state_c]
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 這裡的decoder_lstm跟訓練時所使用的layer是同一個
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

# 要注意inference時，input的形狀跟上面訓練的model的形狀是不一樣的
# 在Model裡面的 + 跟數學運算的 + 性質不同
# 比較像是概念上的 這個 + 那個 (Ex: 給我一個炒麵+蛋)
# Model裡的+表示有輛ˇ個參數要輸入(字母表和initial_state)
decoder_model = Model(
    inputs=[decoder_inputs] + decoder_states_inputs,
    outputs=[decoder_outputs] + decoder_states)

reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # 根據encoder_model的宣告，他輸出的是encoder這個LSTM layer的[state_h, state_c]
    states_value = encoder_model.predict(input_seq)
    
    # 產生一個長度為1的空的 target sequence
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # 設置target_seq的第一個字為起始字元(\t)
    target_seq[0, 0, target_token_index['\t']] = 1.

    # 下面會透過迴圈，繼續對sequence做取樣
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # 輸入字母表: [target_seq] 和 initial_state: states_value
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # 取得一個字元，一個sequence輸入後，輸出也是一個sequence，下面的
        # 下面的-1是用來取得model輸出的sequence的最後一個step的資料。(其他捨棄)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        # 迴圈的離開條件: 到達最長的sequence長度或是出現結束字元
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # 更新 target sequence，在字母表上把取得的字元的對應位置設為1
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # 更新 hidden state的狀態
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # 取得一個sequence(從training set裡面取得)
    # 嚐試decoding
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    # 輸入一筆資料，輸入資料有16列，每一列有70行 (16個字元，每個字元都來自於70種可能的字元之一)
    print(np.array(input_seq).shape)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

(1, 16, 70)
-
Input sentence: Go.
Decoded sentence: Va !

(1, 16, 70)
-
Input sentence: Hi.
Decoded sentence: Salut !

(1, 16, 70)
-
Input sentence: Hi.
Decoded sentence: Salut !

(1, 16, 70)
-
Input sentence: Run!
Decoded sentence: Cours !

(1, 16, 70)
-
Input sentence: Run!
Decoded sentence: Cours !

(1, 16, 70)
-
Input sentence: Who?
Decoded sentence: Qui ?

(1, 16, 70)
-
Input sentence: Wow!
Decoded sentence: Ça alors !

(1, 16, 70)
-
Input sentence: Fire!
Decoded sentence: Au feu !

(1, 16, 70)
-
Input sentence: Help!
Decoded sentence: À l'aide !

(1, 16, 70)
-
Input sentence: Jump.
Decoded sentence: Saute.

(1, 16, 70)
-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

(1, 16, 70)
-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

(1, 16, 70)
-
Input sentence: Stop!
Decoded sentence: Arrête-toi !

(1, 16, 70)
-
Input sentence: Wait!
Decoded sentence: Attends !

(1, 16, 70)
-
Input sentence: Wait!
Decoded sentence: Attends !

(1, 16, 70)
-
Input sentence: Go on.
Decoded 